# film_baseline
To get the general feel for a film, we'll define some baseline characteristics. Some of these, such as conversation speed, can be compared to individual scenes; this helps us find scenes which might be particularly dramatic or emotional.

In [2]:
import sys
sys.path.append('../data_serialization')
from serialization_preprocessing_io import *

### Disregarding credits
First, we'll need a way to disregard frames that contain the credits. This will throw off our baseline characteristics.

As a quick-and-dirty way of accomplishing this, we can look for the last frame that has a face 